<a href="https://colab.research.google.com/github/chiao-lyc/B0729062_CompOrg/blob/master/HW%231.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hw1-dataset.txt to hw1-dataset.txt


In [ ]:
import pandas as pd

read_file = pd.read_csv (r'hw1-dataset.txt',error_bad_lines=False,encoding='utf-8')
read_file.to_csv (r'hw1-dataset-2.csv', index=None)

b'Skipping line 74: expected 1 fields, saw 2\nSkipping line 228: expected 1 fields, saw 2\nSkipping line 248: expected 1 fields, saw 2\nSkipping line 290: expected 1 fields, saw 2\nSkipping line 364: expected 1 fields, saw 2\nSkipping line 519: expected 1 fields, saw 2\nSkipping line 540: expected 1 fields, saw 2\nSkipping line 591: expected 1 fields, saw 2\nSkipping line 670: expected 1 fields, saw 2\nSkipping line 754: expected 1 fields, saw 3\nSkipping line 937: expected 1 fields, saw 2\nSkipping line 1022: expected 1 fields, saw 2\nSkipping line 1101: expected 1 fields, saw 2\nSkipping line 1261: expected 1 fields, saw 2\nSkipping line 1299: expected 1 fields, saw 2\nSkipping line 1335: expected 1 fields, saw 2\nSkipping line 1484: expected 1 fields, saw 2\nSkipping line 1492: expected 1 fields, saw 2\nSkipping line 1495: expected 1 fields, saw 3\nSkipping line 1501: expected 1 fields, saw 2\nSkipping line 1584: expected 1 fields, saw 2\nSkipping line 1644: expected 1 fields, saw 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import re
import jieba
#%%    
class WordSegment:
    def __init__(self,texts,stop_words_path,dicts_path=""):
        self.jieba = jieba
        self.texts = texts
        self.dicts_path = dicts_path
        self.stop_words_path = stop_words_path
        
    def read_dictionary(self): 
        #讀取類別
        print("read_dictionary is called")
        if self.dicts_path:
            for dic in self.dicts_path:
                self.jieba.load_userdict(dic)

    def read_stop_words(self):
        #讀取stopword
        self.stop_words = set()
        for stop_words_file in self.stop_words_path:
            with open(stop_words_file,encoding = 'UTF-8') as file:
                self.stop_words = self.stop_words | set(map(str.strip,file.readlines()))
        
    def remove_stop_words(self):
        # 去除繁體中文以外的英文、數字、符號
        rule = re.compile(r"[^\u4e00-\u9fa5]")
        self.texts = [list(self.jieba.cut(rule.sub('', text))) for text in self.texts]
        for idx, speech in enumerate(self.texts):
            self.texts[idx] = ' '.join([word for word in speech if word.strip() not in self.stop_words])

In [ ]:
import math
import os
from wordcloud import WordCloud
from WordSegment import WordSegment
#%%
storage_folder = "clean data"
filenames = os.listdir(storage_folder)
singers = {}
for filename in filenames:
    path = f"{storage_folder}/{filename}"
    with open(path,encoding="utf-8") as file:
        #句子以字串形式連接
        lyric = ""
        for i,line in enumerate(file.readlines()):
            if i==0:continue
            title, content = line.split(",",1)
            lyric += content
        singer = filename.split(".")[0]
        singers[singer] = lyric

stop_words_path = "stop_words"
stop_words_path = [f"{stop_words_path}/{path}" for path in os.listdir(stop_words_path)]
ws = WordSegment(singers.values(),stop_words_path=stop_words_path)
ws.read_dictionary()
ws.read_stop_words()
ws.remove_stop_words()
text_files = [text.split(" ") for text in ws.texts]
#%% 
#計算tf-idf

#統計每個語詞的次數
words_count = []
for file in text_files:
    count = {}
    for word in file:
        if word in count:
            count[word] += 1
        else:
            count[word] = 1
    words_count.append(count)

# 統計每個語詞的頻率(次數/全部單語詞次數)
words_frequency = []
for word_count in words_count:
    all_count = sum(word_count.values()) #單篇歌詞的所有單詞數量
    fre = {}
    for word,count in word_count.items():
        fre[word] = round(count/all_count,4)
    words_frequency.append(fre)

#idf
#先取得每個語詞在歌手歌詞中"出現過"的次數
all_words = []
for word in words_count:
    all_words.extend(list(word.keys()))
    
occurrences_of_word = {}
for word in all_words:
    if word in occurrences_of_word:
        occurrences_of_word[word] += 1
    else:
        occurrences_of_word[word] = 1
        
inverse_document_frequency = []     
for word_count in words_count:
    #出現過的次數
    invFre = {} 
    for word in word_count.keys():
        occurrences = occurrences_of_word[word]
        invFre[word] = math.log(round((len(words_count)/occurrences),4))
    inverse_document_frequency.append(invFre)
    
##tf*idf
all_tf_idf = []
for i,words in enumerate(words_frequency):
    tf_idf = {}
    for word,freq in words.items():
        tf_idf[word] = freq*inverse_document_frequency[i][word]
    all_tf_idf.append(tf_idf)
